### 01-AI-Nutritionist-Demo

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print('connected to a GPU')
  print(gpu_info)

Sat May 25 07:13:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   39C    P8              4W /   80W |     627MiB /   8188MiB |     50%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import os
import chromadb
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, ServiceContext, get_response_synthesizer, Settings, PromptTemplate
from llama_index.readers.file import PDFReader, CSVReader
from IPython.display import display, Markdown
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.embeddings.openai import OpenAIEmbedding

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LLAMA_INDEX_API_KEY = os.getenv("LLAMA_INDEX_API_KEY")

In [6]:
import chromadb
print("chromadb imported successfully!")

chromadb imported successfully!


In [7]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from llama_parse import LlamaParse
    
# Initialize the LlamaParse object with parameters
parser = LlamaParse(
    result_type="markdown",
    api_key=LLAMA_INDEX_API_KEY,
    verbose=True,
    language="en",
    num_workers=8
)

In [9]:
# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}

reader = SimpleDirectoryReader(input_dir =r"D:\Generative-AI\03-Generative-AI-Frameworks\data\aiNutritionist\Clinical_Reports", file_extractor=file_extractor)
clinicalReports = reader.load_data()

reader = SimpleDirectoryReader(input_dir =r"D:\Generative-AI\03-Generative-AI-Frameworks\data\aiNutritionist\Diet_Plan", file_extractor=file_extractor)
dietPlan = reader.load_data()

reader = SimpleDirectoryReader(input_dir =r"D:\Generative-AI\03-Generative-AI-Frameworks\data\aiNutritionist\ICMR_Guidelines", file_extractor=file_extractor)
icmrGuidelines = reader.load_data()




Started parsing the file under job_id 377422fa-27a3-491b-8009-04b983ff4c2b
Started parsing the file under job_id 662180ea-ffd1-4a98-8abc-5516011cb6f6
Started parsing the file under job_id 4ce81e35-dfa0-4efd-97b4-75e892aa1eec
Started parsing the file under job_id e78eceba-aab2-4918-b47a-e07610aa5e85
Started parsing the file under job_id 9e756a7d-8d32-4a02-8f4a-ca1b3e307a5f
Started parsing the file under job_id 4790a3c6-a5a1-4010-bfc4-7ba9e1fe706f
......

In [10]:
documents= clinicalReports + dietPlan + icmrGuidelines

In [11]:
for i in range(len(documents)):  # Iterate through documents
    chunk_size = len(documents[i].text[0:50000])  
    print(f"Chunk size (characters) for document {i}: {chunk_size}")

Chunk size (characters) for document 0: 22
Chunk size (characters) for document 1: 22
Chunk size (characters) for document 2: 109
Chunk size (characters) for document 3: 49
Chunk size (characters) for document 4: 7289
Chunk size (characters) for document 5: 25000


In [12]:
from llama_index.embeddings.openai import OpenAIEmbedding
# Load global settings
#Settings.api_key = OPENAI_API_KEY

# Load in a specific embedding model
embed_model = OpenAIEmbedding(model="text-embedding-3-large", chunk_size=1024, chunk_overlap=200, embed_batch_size=32,api_key=OPENAI_API_KEY)

# Load in a specific language model
llm = OpenAI(model="gpt-4o", api_key=OPENAI_API_KEY,temperature=0.7, max_tokens=3000, top_p=1, logprobs=True, echo=True, stream=True)


# Load global settings
Settings.embed_model = embed_model
Settings.llm = llm

In [16]:
# Create Custom Prompt Template

template = (
    "You are NutriGuide, an AI assistant specializing in personalized nutrition advice. You provide diet plans and nutritional guidance based on ICMR guidelines and individual health reports. Start each response with 'NutriGuide:'.\n"
    "**Context Information**\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "**User Question:** {query_str}\n"
)


context_str = (
    "User Health Report Summary:\n"
    "- Condition: High blood pressure\n"
    "- Age: 34\n"
    "- Gender: Female\n"
    "- Weight: 74 kg\n"
    "- Height: 160 cm\n"
    "- Lifestyle: Sedentary\n"
    "- Specific dietary restrictions: None\n"
)


qa_template = PromptTemplate(template)


In [14]:
# initialize client
db = chromadb.PersistentClient(path="./chroma_db")

# get collection
chroma_collection = db.get_or_create_collection("quickstart")

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, llm=llm
)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=15,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [17]:
query_engine = index.as_query_engine(text_qa_template=qa_template)
response = query_engine.query("Can you suggest a diet plan to manage my weight loss journey?")
display(Markdown(f"{response}"))

**NutriGuide:**

Certainly! Based on the context provided, here is a diet plan tailored to manage your weight loss journey, ensuring it aligns with the ICMR guidelines:

### **Daily Diet Plan for Weight Loss**

#### **Morning (6:00 AM)**
- **Warm Lemon Water:** 1 glass (without sugar)

#### **Breakfast (8:00 - 9:00 AM)**
- **Option 1:**  
  - Soaked & boiled whole grains (like oats or quinoa) - 60g
  - Boiled chickpea or black beans - 30g
  - Green leafy vegetables (spinach, kale) - 50g
  - Mixed vegetables - 100g
  - Nuts (almonds, walnuts) - 20g
- **Option 2:**  
  - Low-fat yogurt - 1 cup
  - Fresh fruit (apple, berries) - 1 serving
  - A handful of nuts and seeds

#### **Mid-Morning Snack (10:30 AM)**
- **Fruit:** 1 whole fruit (avoid fruit juices)

#### **Lunch (1:00 - 2:00 PM)**
- **Option 1:**  
  - Whole grain cereals (brown rice/whole wheat chapati) - 80g
  - Pulses (dal) - 20g
  - Mixed vegetable curry - 150g
  - Green leafy vegetables - 50g
  - Low-fat curd - 150ml
  - End meal with a small fruit serving (50g)
- **Option 2:**  
  - Vegetable khichdi - 1 cup
  - Cucumber salad - 1 cup
  - Low-fat curd - 1 cup

#### **Afternoon (4:00 PM)**
- **Buttermilk:** 1 glass (without sugar)

#### **Evening Beverage (5:00 PM)**
- **Milk:** 50ml (skimmed milk)
- **Nuts/Seeds:** A handful of mixed nuts or seeds

#### **Dinner (7:00 - 8:00 PM)**
- **Option 1:**  
  - Whole grain cereals (whole wheat chapati) - 60g
  - Pulses (dal) - 15g
  - Mixed vegetables - 50g
  - Low-fat curd - 100ml
  - End with a small fruit serving (50g)
- **Option 2:**  
  - Poli (whole wheat chapati) - 2 pieces
  - Grilled chicken - 1 piece
  - Cucumber tomato salad - 1 bowl

#### **Late Evening (10:00 PM)**
- **Skimmed milk with turmeric:** 1 glass (without sugar)

### **Additional Guidelines:**
- **Hydration:** Drink at least 3 liters of water daily.
- **Healthy Fats:** Include flaxseeds, nuts like almonds, and walnuts. Opt for oils low in saturated fats like mustard oil or sunflower oil.
- **Avoid:** Sugary foods, refined flours, bakery products, fried/junk food, high-fat sugar products, carbonated drinks, and packaged foods.
- **Salads:** Always start your meals with a salad.
- **Physical Activity:** Engage in regular physical activity and yoga to support weight loss.

### **Notes:**
- **Portion Control:** Ensure the portion sizes are appropriate to avoid excessive calorie intake.
- **Meal Timing:** Maintain consistent meal timings to support metabolism.
- **Variety:** Incorporate a variety of fruits and vegetables to meet nutritional requirements.

This diet plan focuses on balanced nutrition, portion control, and the inclusion of whole foods to support gradual and sustainable weight loss. Always consult with a healthcare provider before starting any new diet plan, especially if you have specific health conditions or dietary needs.

If you need further modifications or have specific preferences, feel free to reach out!

**NutriGuide:** Wishing you a healthy and successful weight loss journey!